In [64]:
from glob import glob
import os
import pickle
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aleksandr_Mishin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Чтение данных в датафрейм:

In [2]:
PATH_TO_TEST_DATA = './test/'
PATH_TO_TRAIN_DATA = './train/'

In [12]:
def prepare_train_set(path_to_files):
    df = pd.DataFrame(columns=['text', 'target'])
    positive_files = glob(path_to_files + "pos/*.txt")
    negative_files = glob(path_to_files + "neg/*.txt")
    
    for file in tqdm(positive_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, True]
    for file in tqdm(negative_files):
        with open(file, 'r', encoding="utf8") as content_file:
            content = content_file.read()
            df.loc[df.shape[0]] = [content, False]
    
    return df

In [16]:
df = prepare_train_set(PATH_TO_TRAIN_DATA)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Сериализуем датафрейм, чтобы не пересчитывать при перезапуске ядра.

In [17]:
with open ("./pickle/df.pkl", 'wb') as df_pkl:
    pickle.dump(df, df_pkl, protocol=2)

In [49]:
df = pickle.load(open("./pickle/df.pkl", "rb"))

In [50]:
pd.set_option('display.max_colwidth', -1)
df.head(3)

,text,target
0,"Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as ""Teachers"". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is ""Teachers"". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!",True
1,"Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days without the luxuries; if Bolt succeeds, he can do what he wants with a future project of making more buildings. The bet's on where Bolt is thrown on the street with a bracelet on his leg to monitor his every move where he can't step off the sidewalk. He's given the nickname Pepto by a vagrant after it's written on his forehead where Bolt meets other characters including a woman by the name of Molly (Lesley Ann Warren) an ex-dancer who got divorce before losing her home, and her pals Sailor (Howard Morris) and Fumes (Teddy Wilson) who are already used to the streets. They're survivors. Bolt isn't. He's not used to reaching mutual agreements like he once did when being rich where it's fight or flight, kill or be killed.<br /><br />While the love connection between Molly and Bolt wasn't necessary to plot, I found ""Life Stinks"" to be one of Mel Brooks' observant films where prior to being a comedy, it shows a tender side compared to his slapstick work such as Blazing Saddles, Young Frankenstein, or Spaceballs for the matter, to show what it's like having something valuable before losing it the next day or on the other hand making a stupid bet like all rich people do when they don't know what to do with their money. Maybe they should give it to the homeless instead of using it like Monopoly money.<br /><br />Or maybe this film will inspire you to help others.",True
2,"Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in Blazing Saddles. The take on lawyers is also superb. After being accused of being a turncoat, selling out his boss, and being dishonest the lawyer of Pepto Bolt shrugs indifferently ""I'm a lawyer"" he says. Three funny words. Jeffrey Tambor, a favorite from the later Larry Sanders show, is fantastic here too as a mad millionaire who wants to crush the ghetto. His character is more malevolent than usual. The hospital scene, and the scene where the homeless invade a demolition site, are all-time classics. Look for the legs scene and the two big diggers fighting (one bleeds). This mov

In [51]:
import string
df.text = df.text.apply(lambda x: x.lower())

tr = str.maketrans("", "", string.punctuation)
df.text = df.text.apply(lambda x: x.translate(tr))


In [52]:
df.head(1)

,text,target
0,bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell highs satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector im here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isnt,True


Убираем стоп слова

In [53]:
from nltk.corpus import stopwords

stoplist = stopwords.words('english')
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))

In [54]:
df.head(3)

,text,target
0,bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,True
1,homelessness houselessness george carlin stated issue years never plan help street considered human everything going school work vote matter people think homeless lost cause worrying things racism war iraq pressuring kids succeed technology elections inflation worrying theyll next end streetsbr br given bet live streets month without luxuries home entertainment sets bathroom pictures wall computer everything treasure see like homeless goddard bolts lessonbr br mel brooks directs stars bolt plays rich man everything world deciding make bet sissy rival jeffery tambor see live streets thirty days without luxuries bolt succeeds wants future project making buildings bets bolt thrown street bracelet leg monitor every move cant step sidewalk hes given nickname pepto vagrant written forehead bolt meets characters including woman name molly lesley ann warren exdancer got divorce losing home pals sailor howard morris fumes teddy wilson already used streets theyre survivors bolt isnt hes used reaching mutual agreements like rich fight flight kill killedbr br love connection molly bolt wasnt necessary plot found life stinks one mel brooks observant films prior comedy shows tender side compared slapstick work blazing saddles young frankenstein spaceballs matter show like something valuable losing next day hand making stupid bet like rich people dont know money maybe give homeless instead using like monopoly moneybr br maybe film inspire help others,True
2,brilliant overacting lesley ann warren best dramatic hobo lady ever seen love scenes clothes warehouse second none corn face classic good anything blazing saddles take lawyers also superb accused turncoat selling boss dishonest lawyer pepto bolt shrugs indifferently im lawyer says three funny words jeffrey tambor favorite later larry sanders show fantastic mad millionaire wants crush ghetto character malevolent usual hospital scene scene homeless invade demolition site alltime classics look legs scene two big diggers fighting one bleeds movie gets better time see quite often,True


Применим PorterStemer на входные данные.

In [67]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

def stem_sentence(sentence):
    porter_stemmer = PorterStemmer()
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

df['stem_text'] = df['text'].apply(stem_sentence)

In [68]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter

def get_pos(word):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0]

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = sentence.split()
    lemmed_tokens = [lemmatizer.lemmatize(token, get_pos(word)) for token in tokens]
    return ' '.join(lemmed_tokens)

df['lemm_text'] = df['text'].apply(lemmatize_sentence)

In [69]:
df.head(1)

,text,target,stem_text,lemm_text
0,bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt,True,bromwel high cartoon comedi ran time program school life teacher 35 year teach profess lead believ bromwel high satir much closer realiti teacher scrambl surviv financi insight student see right pathet teacher pomp petti whole situat remind school knew student saw episod student repeatedli tri burn school immedi recal high classic line inspector im sack one teacher student welcom bromwel high expect mani adult age think bromwel high far fetch piti isnt,bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt


Применим аналогичные операции для отложенной выборки:

In [96]:
test_df = prepare_train_set(PATH_TO_TEST_DATA)
test_df.text = test_df.text.apply(lambda x: x.lower())
test_df.text = test_df.text.apply(lambda x: x.translate(tr))
test_df.text = test_df.text.apply(lambda x: ' '.join([word for word in x.split() if word not in stoplist]))
test_df['stem_text'] = test_df['text'].apply(stem_sentence)
test_df['lemm_text'] = test_df['text'].apply(lemmatize_sentence)

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

HBox(children=(IntProgress(value=0, max=12500), HTML(value='')))

Применим bag-of-words для получения фич по токенам:

In [88]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow = vectorizer.fit_transform(df.text[:10])

In [89]:
bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 1, 1, 0],
        [1, 0, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [2]:
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.naive_bayes import MultinomialNB

In [97]:
count_vectorizer = CountVectorizer(max_features=50000)
model = linear_model.SGDClassifier()

In [98]:
X = df['lemm_text']
y = df['target']
y=y.astype('bool')

X_bow = count_vectorizer.fit_transform(X)
model.fit(X_bow, y)

C:\Users\Aleksandr_Mishin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [102]:
X_test_bow = count_vectorizer.transform(test_df.text)
y_test = test_df.target
y_test=y_test.astype('bool')


accuracy_score(y_test, model.predict(X_test_bow))

0.80896

In [136]:
coefs_dict = dict(zip(count_vectorizer.get_feature_names(), model.coef_[0]))

In [137]:
import operator
sorted_dict = sorted(coefs_dict.items(), key=operator.itemgetter(1), reverse=True)

In [141]:
sorted_dict[:15]

[('710', 6.587353867887816),
 ('refreshing', 4.047651171834686),
 ('810', 3.6508226255763856),
 ('kitty', 3.5714569163247174),
 ('wonderfully', 3.333359788569741),
 ('subtle', 3.253994079318085),
 ('1010', 3.174628370066429),
 ('rare', 2.936531242311433),
 ('gandhi', 2.8571655330597836),
 ('favorite', 2.857165533059783),
 ('highly', 2.777799823808128),
 ('superb', 2.7777998238081247),
 ('driven', 2.7777998238081145),
 ('hooked', 2.7777998238081105),
 ('funniest', 2.698434114556469)]

In [150]:
def train_and_predict_bow(estimator, ngrams=(1,1),  column='text'):
    count_vectorizer = CountVectorizer(max_features=50000, ngram_range=ngrams)
    
    X_train = df[column]
    y_train = df['target']
    y_train = y_train.astype('bool')
    X_train_bow = count_vectorizer.fit_transform(X_train)
    estimator.fit(X_train_bow, y_train)  
    
    X_test_bow = count_vectorizer.transform(test_df.text)
    y_test = test_df.target
    y_test=y_test.astype('bool')


    print("accuracy: " + str(accuracy_score(y_test, estimator.predict(X_test_bow))))
    print("roc auc: " + str(roc_auc_score(y_test, estimator.predict(X_test_bow))))

In [145]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='stem_text')

C:\Users\Aleksandr_Mishin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


accuracy: 0.75508
roc auc: 0.75508


In [1]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='lemm_text')

NameError: name 'train_and_predict_bow' is not defined

In [147]:
train_and_predict_bow(estimator=linear_model.SGDClassifier(), column='text')

C:\Users\Aleksandr_Mishin\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


accuracy: 0.838
roc auc: 0.8380000000000002


In [ ]:
train_and_predict_bow(estimator=svm.SVC(), column='lemm_text')

In [ ]:
train_and_predict_bow(estimator=MultinomialNB(), column='lemm_text')